In [ ]:
# !pip install transformers[torch] datasets evaluate

In [ ]:
# RUNWAY_DATA_PATH는 Link Pipeline Parameters로 등록 되어 있어야 합니다.
import pandas as pd

df = pd.read_parquet(RUNWAY_DATA_PATH)

In [ ]:
from datasets import Dataset

ds = Dataset.from_pandas(df.head(100))
ds.set_format("pt")

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer = AutoTokenizer.from_pretrained(MODEL_ARCH_NAME)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_ds = ds.map(preprocess_function, batch_size=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

device = "cuda" if torch.cuda.is_available() else "cpu"
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ARCH_NAME, num_labels=2, id2label=id2label, label2id=label2id
).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="tmp",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/home/jovyan/.local/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=25, training_loss=0.3085244369506836, metrics={'train_runtime': 2.6072, 'train_samples_per_second': 38.355, 'train_steps_per_second': 9.589, 'total_flos': 11448080968224.0, 'train_loss': 0.3085244369506836, 'epoch': 1.0})

In [ ]:
import pandas as pd


class HuggingModel:
    def __init__(self, pipeline):
        self.pipeline = pipeline
    
    def predict(self, X):
        result = self.pipeline(X["text"].to_list())
        return pd.DataFrame.from_dict(result)

In [ ]:
from transformers import pipeline


model = model.to("cpu")
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

hug_model = HuggingModel(pipe)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
input_sample = df.sample(1).drop(columns=["label"])

In [ ]:
import runway

runway.log_model(model_name='my-model', model=hug_model, input_samples={'predict': input_sample})


/usr/local/lib/python3.9/dist-packages/runway/common/post_save_utils.py:89: UserWarning: model name is changed from my-model to my-model-f8a9fec1
  warnings.warn(
